<a href="https://colab.research.google.com/github/Dhruvit-Agrawal/PyTorch/blob/main/5_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [ ]:
torch.manual_seed(42)

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/PyTorch/Fashion MNIST/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [ ]:
#train test split
x=df.drop('label',axis=1)
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2)

In [ ]:
#scaling
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
#custom dataset and dataLoader classes
from torch.utils.data import Dataset, DataLoader

In [ ]:
#custom dataset
class FashionDataset(Dataset):
  def __init__(self,x,y):
    self.x=torch.tensor(x,dtype=torch.float32)   # features in float
    self.y=torch.tensor(y,dtype=torch.long)      # labels in 'long'

  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]

In [ ]:
#creating datasets
train_dataset=FashionDataset(x_train,y_train)
test_dataset=FashionDataset(x_test,y_test.values)

In [ ]:
#creating data loaders
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True, pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False, pin_memory=True)

In [ ]:
## neural network

class MyNNModel(nn.Module):
  def __init__(self,input_size):

    super().__init__()
    self.network= nn.Sequential(
        nn.Linear(in_features=input_size, out_features=128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10),
        nn.LogSoftmax(dim=1)
    )
  def forward(self,x):
    return self.network(x)


In [ ]:
input_size=x_train.shape[1]
#mdel instance
model=MyNNModel(input_size)

#moving to GPU
model.to(device)

MyNNModel(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
)

In [ ]:
#parameters
epochs=100
learning_rate=0.01


#loss function
loss_fn=nn.CrossEntropyLoss()

#optimizer
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
#training

for epoch in range(epochs):

  for batch_x,batch_y in train_loader:    #for each batch

    #moving to gpu
    batch_x=batch_x.to(device)
    batch_y=batch_y.to(device)

    #forward
    y_hat=model(batch_x)

    #zeoring gradients
    optimizer.zero_grad()

    #loss
    loss=loss_fn(y_hat,batch_y)

    #backward
    loss.backward()

    #update weights
    optimizer.step()

  print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.8061256408691406
Epoch: 2, Loss: 0.8775932788848877
Epoch: 3, Loss: 0.5470736622810364
Epoch: 4, Loss: 0.3914673924446106
Epoch: 5, Loss: 0.8779534101486206
Epoch: 6, Loss: 0.739471435546875
Epoch: 7, Loss: 0.5901367664337158
Epoch: 8, Loss: 0.466350257396698
Epoch: 9, Loss: 0.6323586702346802
Epoch: 10, Loss: 0.2450941950082779
Epoch: 11, Loss: 0.32901307940483093
Epoch: 12, Loss: 0.5087697505950928
Epoch: 13, Loss: 0.6943427324295044
Epoch: 14, Loss: 0.4247514009475708
Epoch: 15, Loss: 0.08684264868497849
Epoch: 16, Loss: 0.3859052360057831
Epoch: 17, Loss: 1.024710774421692
Epoch: 18, Loss: 0.36986565589904785
Epoch: 19, Loss: 0.35702210664749146
Epoch: 20, Loss: 0.3635322153568268
Epoch: 21, Loss: 0.9362584948539734
Epoch: 22, Loss: 0.9739224910736084
Epoch: 23, Loss: 0.12464798241853714
Epoch: 24, Loss: 0.17592041194438934
Epoch: 25, Loss: 0.28849199414253235
Epoch: 26, Loss: 0.3611096143722534
Epoch: 27, Loss: 0.227901428937912
Epoch: 28, Loss: 0.150723502039909

In [ ]:
#evaluation mode
model.eval()

MyNNModel(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
)

In [ ]:
#prediction
accuracy=[]

with torch.no_grad(): #switching off the history
  for batch_x,batch_y in test_loader:   #for each epoch

    #moving to gpu
    batch_x=batch_x.to(device)
    batch_y=batch_y.to(device)

    #prediction
    y_pred_pvalues=model(batch_x)
    y_pred=torch.argmax(y_pred_pvalues,dim=1)

    #accuracy
    batch_accuracy= (y_pred==batch_y).float().mean()
    print(f"BatchAccuracy: {batch_accuracy.item()}")
    accuracy.append(batch_accuracy.item())

  avg_accuracy= np.mean(accuracy)
  print(f"Average Accuracy: {avg_accuracy}")




BatchAccuracy: 0.71875
BatchAccuracy: 0.84375
BatchAccuracy: 0.625
BatchAccuracy: 0.6875
BatchAccuracy: 0.6875
BatchAccuracy: 0.8125
BatchAccuracy: 0.8125
BatchAccuracy: 0.8125
BatchAccuracy: 0.78125
BatchAccuracy: 0.71875
BatchAccuracy: 0.75
BatchAccuracy: 0.8125
BatchAccuracy: 0.65625
BatchAccuracy: 0.78125
BatchAccuracy: 0.84375
BatchAccuracy: 0.75
BatchAccuracy: 0.59375
BatchAccuracy: 0.8125
BatchAccuracy: 0.84375
BatchAccuracy: 0.65625
BatchAccuracy: 0.75
BatchAccuracy: 0.78125
BatchAccuracy: 0.78125
BatchAccuracy: 0.71875
BatchAccuracy: 0.75
BatchAccuracy: 0.71875
BatchAccuracy: 0.8125
BatchAccuracy: 0.78125
BatchAccuracy: 0.84375
BatchAccuracy: 0.8125
BatchAccuracy: 0.84375
BatchAccuracy: 0.6875
BatchAccuracy: 0.65625
BatchAccuracy: 0.6875
BatchAccuracy: 0.65625
BatchAccuracy: 0.8125
BatchAccuracy: 0.8125
BatchAccuracy: 0.875
Average Accuracy: 0.7574013157894737
